![walmartecomm](walmartecomm.jpg)

Walmart is the biggest retail store in the United States. Just like others, they have been expanding their e-commerce part of the business. By the end of 2022, e-commerce represented a roaring $80 billion in sales, which is 13% of total sales of Walmart. One of the main factors that affects their sales is public holidays, like the Super Bowl, Labour Day, Thanksgiving, and Christmas. 

In this project, you have been tasked with creating a data pipeline for the analysis of supply and demand around the holidays, along with conducting a preliminary analysis of the data. You will be working with two data sources: grocery sales and complementary data. You have been provided with the `grocery_sales` table in `PostgreSQL` database with the following features:

# `grocery_sales`
- `"index"` - unique ID of the row
- `"Store_ID"` - the store number
- `"Date"` - the week of sales
- `"Weekly_Sales"` - sales for the given store

Also, you have the `extra_data.parquet` file that contains complementary data:

# `extra_data.parquet`
- `"IsHoliday"` - Whether the week contains a public holiday - 1 if yes, 0 if no.
- `"Temperature"` - Temperature on the day of sale
- `"Fuel_Price"` - Cost of fuel in the region
- `"CPI"` – Prevailing consumer price index
- `"Unemployment"` - The prevailing unemployment rate
- `"MarkDown1"`, `"MarkDown2"`, `"MarkDown3"`, `"MarkDown4"` - number of promotional markdowns
- `"Dept"` - Department Number in each store
- `"Size"` - size of the store
- `"Type"` - type of the store (depends on `Size` column)

You will need to merge those files and perform some data manipulations. The transformed DataFrame can then be stored as the `clean_data` variable containing the following columns:
- `"Store_ID"`
- `"Month"`
- `"Dept"`
- `"IsHoliday"`
- `"Weekly_Sales"`
- `"CPI"`
- "`"Unemployment"`"

After merging and cleaning the data, you will have to analyze monthly sales of Walmart and store the results of your analysis as the `agg_data` variable that should look like:

|  Month | Weekly_Sales  | 
|---|---|
| 1.0  |  33174.178494 |
|  2.0 |  34333.326579 |
|  ... | ...  |  

Finally, you should save the `clean_data` and `agg_data` as the csv files.

It is recommended to use `pandas` for this project. 

In [4]:
import pandas as pd
import os

In [7]:
grocery_sales = pd.read_csv(r'C:\Users\Azamat.ma\Desktop\ETL\main\datalab_export_2024-08-04 12_14_37.csv')

In [20]:
import pandas as pd
import os

# Start here...

def extract(sales_date, extra_data):
    extra_df = pd.read_parquet(extra_data)
    merged_df = sales_date.merge(extra_df, on='index')
    return merged_df

merged_df = extract(grocery_sales, "extra_data.parquet")


def transform(raw_data):
    raw_data.fillna(
    {
        'CPI':raw_data['CPI'].mean(),
        'Weekly_Sales': raw_data['Weekly_Sales'].mean(),
        'Unemployment': raw_data['Unemployment'].mean()
    }, inplace=True
    )
    
    raw_data['Date'] = pd.to_datetime(raw_data['Date'], format='%Y-%m-%d')
    raw_data['Month'] = raw_data['Date'].dt.month  # Add this line to extract the month
    
    raw_data = raw_data.loc[raw_data["Weekly_Sales"] > 10000, :]
    
    raw_data = raw_data.drop(["index", "Temperature", "Fuel_Price", "MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5", "Type", "Size", "Date"], axis = 1)
    return raw_data

clean_data = transform(merged_df)


def avg_monthly_sales(clean_data):
    holiday_sales = clean_data[['Month', 'Weekly_Sales']]
    
    holiday_sales = (holiday_sales.groupby('Month').agg(Avg_Sales = ('Weekly_Sales', 'mean')).reset_index().round(2))
    return holiday_sales

agg_data = avg_monthly_sales(clean_data)
print(agg_data)

def load(full_data, full_data_file_path, agg_data, agg_data_file_path):
    full_data.to_csv(full_data_file_path, index=False)
    agg_data.to_csv(agg_data_file_path, index=False)
    
load(clean_data, "clean_data.csv", agg_data, "agg_data.csv")


def validation(file_path):
  	# Use the "os" package to check whether a path exists
    file_exists = os.path.exists(file_path)
    # Raise an exception if the path doesn't exist, hence, if there is no file found on a given path
    if not file_exists:
        raise Exception(f"There is no file at the path {file_path}")

# Call the validation() function and pass first, the cleaned DataFrame path, and then the aggregated DataFrame path
validation("clean_data.csv")
validation("agg_data.csv")

    Month  Avg_Sales
0     1.0   33174.18
1     2.0   34333.33
2     3.0   33220.89
3     4.0   33392.37
4     5.0   33339.89
5     6.0   34582.47
6     7.0   33922.76
7     8.0   33644.79
8     9.0   33258.05
9    10.0   32736.99
10   11.0   36594.03
11   12.0   39238.80
